# Basic Setup

Install and import some libraries.

In [52]:
# Imports
import pandas as pd
import re
import random
import copy
import math
import json
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from collections import Counter
import statsmodels.api as sm
from scipy.stats import linregress

# Import The IPA dictionary

This section of the code imports the IPA dictionary used to generate the various shorthand abbreviation systems.

In [53]:
# Scoring methods designed to select one of the options from the IPA dictionary.
# It selects the shortest words with the most schwas (laziest pronunciation).


def score(word):
    word.replace(" ", "")
    return (len(word), -word.count("ə"))


def get_laziest(ipa_list):
    return list(sorted(ipa_list, key=score))[0]


# Loading the dictionary and parsing out the best scoring pronunciation
ipa = pd.read_csv("en_US.txt", sep='\t', names=["eng", "ipa"])
ipa_opt_dict = {row[1]['eng']: row[1]['ipa'].split(', ') for row in ipa.iterrows()}
ipa_dict = {row[1]['eng']: get_laziest(row[1]['ipa'].split(', ')) for row in ipa.iterrows()}

# Translation code

This code will now translate using various rules for abbreviaiont.  The code should be examined for details, but I wrote small snippets of rules to define a few different ways to transform from my folk phonetic into various abbreviated forms.  

In [54]:
# Code to do basic string transformations
def translate_add(string, rules, verbose = False): 
    out = []
    pos = 0
    while pos < len(string):
        matched = False
        for rule_head, rule_tail in rules:
            if string[pos:pos+len(rule_head)] == rule_head:
                if rule_tail != None:
                    out.append(rule_tail)
                matched = True
                pos += len(rule_head)
                if verbose:
                    print(f"Matched: {rule_head} -> {rule_tail}")
                break
        if not matched:
            if verbose:
                print(f"Skipped: {string[pos]}")
            pos += 1
    return out if len(out) > 0 else ["e"]

In [55]:
# Consonants without variable representation
unpaired_consonants = [('ɫ', 'l'),
 ('ɹ', 'r'),
 ('m', 'm'),
 ('ŋ', 'ŋ'),
 ('n', 'n'),
 ('w', 'w'),
 ('h', 'h'),
 ('j', 'y')]

# ch and j
full_affrictives = [
 ('dʒ', 'j'),
 ('tʃ', 'C')]

merged_affrictives = [
 ('dʒ', 'C'),
 ('tʃ', 'C')]

# Frictives
full_frictives = [
 ('s', 's'),
 ('ʒ', 'ʒ'),
 ('ʃ', 'ʃ'),
 ('v', 'v'),
 ('θ', 'θ'),
 ('f', 'f'),
 ('z', 'z'),
 ('ð', 'ð')]

merged_frictives = [
 ('s', 's'),
 ('ʒ', 'ʃ'),
 ('ʃ', 'ʃ'),
 ('v', 'f'),
 ('θ', 'θ'),
 ('f', 'f'),
 ('z', 's'),
 ('ð', 'θ')]

# Plosives
full_plosives = [
 ('t', 't'),
 ('b', 'b'),
 ('ɡ', 'g'),
 ('p', 'p'),
 ('d', 'd'),
 ('k', 'k')]

merged_plosives = [
 ('t', 't'),
 ('b', 'p'),
 ('ɡ', 'k'),
 ('p', 'p'),
 ('d', 't'),
 ('k', 'k')]

consonant_types = {
    'full consonants': full_affrictives + unpaired_consonants + full_frictives + full_plosives,
    'full plosives, merged frictives': full_affrictives + unpaired_consonants + merged_frictives + full_plosives,
    'merged consonants': merged_affrictives + unpaired_consonants + merged_frictives + merged_plosives,
}

short_consonant_names = {
    'full consonants': 'FC',
    'full plosives, merged frictives': 'FPMF',
    'merged consonants': 'MC',
}

In [56]:
# Schwa's Name
schwa_name = 'e'

# Stupid R
r_with_vowels = [('ɝ', schwa_name + 'r')]
r_with_lateral_vowels = [('~ɝ', schwa_name + 'r'),('ɝ', 'r')]
r_with_flattened_lateral_vowels = [('~ɝ', 'er'),('ɝ', 'r')]
r_without_vowels = [('ɝ', 'r')]

In [57]:
# Long Vowels
long_vowels = [
 ('i', 'e'),
 ('eɪ', 'a'),
 ('oʊ', 'o'),
 ('aɪ', 'i'),
 ('ju', 'u'),
]

lateral_long_vowels = [
 ('~i', 'e'),
 ('~eɪ', 'a'),
 ('~oʊ', 'o'),
 ('~aɪ', 'i'),
 ('~ju', 'u'),
 ('i~', 'e'),
 ('eɪ~', 'a'),
 ('oʊ~', 'o'),
 ('aɪ~', 'i'),
 ('ju~', 'u'),
]

flattened_lateral_long_vowels = [
 ('~i', 'e'),
 ('~eɪ', 'e'),
 ('~oʊ', 'e'),
 ('~aɪ', 'e'),
 ('~ju', 'e'),
 ('i~', 'e'),
 ('eɪ~', 'e'),
 ('oʊ~', 'e'),
 ('aɪ~', 'e'),
 ('ju~', 'e'),
]

# Schwa
schwa = [
 ('ə', schwa_name),
]

lateral_schwa = [
 ('~ə', schwa_name),
 ('ə~', schwa_name),
]

flattened_lateral_schwa = [
 ('~ə', 'e'),
 ('ə~', 'e'),
]

# remaining vowels
remaining_vowels = [
 ('ɑ', 'a'),
 ('æ', 'a'),
 ('e', 'e'),
 ('ʊ', 'u'),
 ('ɛ', 'e'),
 ('ɔ', 'o'),
 ('a', 'a'),
 ('o', 'o'),
 ('ɪ', 'i'),
 ('u', 'u'),
]

lateral_remaining_vowels = [
 ('~ɑ', 'a'),
 ('~æ', 'a'),
 ('~e', 'e'),
 ('~ʊ', 'u'),
 ('~ɛ', 'e'),
 ('~ɔ', 'o'),
 ('~a', 'a'),
 ('~o', 'o'),
 ('~ɪ', 'i'),
 ('~u', 'u'),
 ('ɑ~', 'a'),
 ('æ~', 'a'),
 ('e~', 'e'),
 ('ʊ~', 'u'),
 ('ɛ~', 'e'),
 ('ɔ~', 'o'),
 ('a~', 'a'),
 ('o~', 'o'),
 ('ɪ~', 'i'),
 ('u~', 'u'),
]

flattened_lateral_remaining_vowels = [
 ('~ɑ', 'e'),
 ('~æ', 'e'),
 ('~e', 'e'),
 ('~ʊ', 'e'),
 ('~ɛ', 'e'),
 ('~ɔ', 'e'),
 ('~a', 'e'),
 ('~o', 'e'),
 ('~ɪ', 'e'),
 ('~u', 'e'),
 ('ɑ~', 'e'),
 ('æ~', 'e'),
 ('e~', 'e'),
 ('ʊ~', 'e'),
 ('ɛ~', 'e'),
 ('ɔ~', 'e'),
 ('a~', 'e'),
 ('o~', 'e'),
 ('ɪ~', 'e'),
 ('u~', 'e'),
]

# kill 'y' as part of long vowels
no_long_u = [('ju','')]

vowel_types = {
    'full vowels': r_with_vowels + long_vowels + schwa + remaining_vowels,
    'schwa suppressed vowels': r_with_lateral_vowels + long_vowels + lateral_schwa + remaining_vowels,
    'short suppressed vowels': r_with_lateral_vowels + long_vowels + lateral_schwa + lateral_remaining_vowels,
    'long vowels': r_without_vowels + long_vowels,
    'lateral vowels': r_with_lateral_vowels + lateral_long_vowels + lateral_schwa + lateral_remaining_vowels + no_long_u,
    'flattened lateral vowels': r_with_flattened_lateral_vowels + flattened_lateral_long_vowels + lateral_schwa + flattened_lateral_remaining_vowels + no_long_u,
    'no vowels': r_without_vowels + no_long_u 
}

short_vowel_names = {
    'full vowels': 'FV',
    'schwa suppressed vowels': 'SSV',
    'short suppressed vowels': 'ShSV',
    'long vowels': 'LoV',
    'lateral vowels': 'LV',
    'flattened lateral vowels': 'FLV',
    'no vowels': 'NV' 
}

In [58]:
def translate(ipa_word, rules, verbose = False):
    temp = "".join(translate_add("~" + ipa_word.replace('/','').replace('ˈ','').replace('ˌ','') + "~",rules, verbose))
    return temp if len(temp) > 0 else "e"

def sentence_translate(ipa_sentence,rules):
    return " ".join([translate(ipa_word,rules) for ipa_word in ipa_sentence.split()])

In [59]:
import string

def remove_punctuation(text):
    """Removes punctuation from a string."""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# The Harvard Sentences

This is an important part: we will use the [Harvard Sentences](https://en.wikipedia.org/wiki/Harvard_sentences).  Thankfully, I had never heard of these before, because if I had, it would have ruined my ability to use them in this experiment.  The Harvard Sentences are 720 grammatically correct sentences on random topics, all of about 10 words each, that were designed to benchmark telephone systems ability to transmit speech. What makes them particularly useful here is that they are designed to be *phonetically balanced*, which is to say that they use the sounds of English in the same frequency as they appear in English.  This makes them fantastic for benchmarking the performance of a phonetic shorthand system.

The experimental protocol is as follows:
1. The system will loop exactly once over all the Harvard Sentences.
2. For each sentence, it will select a random abbreviation system from the above methods.
3. The choice of system, along with the encoded sentence, is displayed to the experiment subject.
4. The subject then types in their translation of the sentence back to standard English.

As I have never seen these sentences before, the only information I have on their content is through the shorthand. Thus, my ability to translate these sentences with the context of the other words around can be compared with the theoretical predictions.

In [60]:
bank = []

with open("harvard-sentences.txt", "r") as file:
    lines = file.readlines()

pos = 0

def harvard_task():
    global pos
    try:
        raw_sentence = lines[pos]
        pos += 1
        ipa_sentence = " ".join(ipa_dict[word][1:-1] for word in remove_punctuation(raw_sentence[:-1]).lower().split())
        consonant_type = random.choice(list(consonant_types.keys()))
        vowel_type = random.choice(list(vowel_types.keys()))

        translated_sentence = sentence_translate(ipa_sentence,vowel_types[vowel_type] + consonant_types[consonant_type])
        return raw_sentence, ipa_sentence, consonant_type, vowel_type, translated_sentence 
    except:
        return "","","","",""


In [61]:
import json
with open('sentence_bank.json', 'r', encoding='utf-8') as f:
    bank = json.load(f)

pos = lines.index(bank[-1][0]) + 1

In [62]:
import ipywidgets as widgets
from IPython.display import display

current = harvard_task()

# Displayed paragraph
paragraph = widgets.HTML(
    value=f"<p><h2>System</h2>{current[2]}<br>{current[3]}<h2>Sample</h2>{current[4]}</p>"
)

# Text input widget (continuous_update is irrelevant here)
text_input = widgets.Text(
    placeholder='Type your response here...',
    description='Input:',
    layout=widgets.Layout(width='50%')
)

# Function called explicitly when Enter is pressed
def on_enter_pressed(sender):
    global current
    if sender.value:
        bank.append((*current, sender.value))
        current = harvard_task()
        paragraph.value = f"<p><h2>System</h2>{current[2]}<br>{current[3]}<h2>Sample</h2>{current[4]}</p>"
        sender.value = ''

# Bind the on_submit callback
text_input.on_submit(on_enter_pressed)

# Display widgets vertically
display(widgets.VBox([paragraph, text_input]))

/var/folders/ng/pntt6yds68xb60yvl8c33wpm0000gq/T/ipykernel_40721/3329733450.py:28: DeprecationWarning:

on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').



# Saving Code

This is currently commented out, but here is some code for saving the sentence banks once gathered.  I've already done it for myself, if you want to try to replicate this experiment on your own, you should delete `sentence_bank.json` and then run throught the widget above, and save it with the commented out code below.

In [63]:
# import json
# with open('sentence_bank.json', 'w', encoding='utf-8') as f:
#     json.dump(bank, f, ensure_ascii=False, indent=4)

# Analysis

We can now analyze and compare with the theoretical predictions from the reconstruction probability.  First, we will compute the estimates of:

1. **The measured error rate.** This is simply the fraction of the words I actually got right.
2. **The measured complexity.** This is an explicit estimate of outline complexity.  It is the $log_2$ of the number of different distinct letters, times the mean length.

I'll also compute some error bars for these, so we can compare them.

After that, we compare the theoretical probabilities with the measured probabilities.  These two will be logit transformed, then a linear fit will be made in the logit space (a fairly standard technique).

In [64]:
right = {}
wrong = {}

for sent in bank:
    if len(remove_punctuation(sent[0].lower()).split()) != len(remove_punctuation(sent[5].lower()).split()):
        print("ERROR:", sent)
        continue
    right[(sent[2],sent[3])] = right.get((sent[2],sent[3]),0) + sum(el1 == el2 for el1, el2 in zip(remove_punctuation(sent[0].lower()).split(), remove_punctuation(sent[5].lower()).split()))
    wrong[(sent[2],sent[3])] = wrong.get((sent[2],sent[3]),0) + sum(el1 != el2 for el1, el2 in zip(remove_punctuation(sent[0].lower()).split(), remove_punctuation(sent[5].lower()).split()))

In [65]:
error_rate = {type:sm.stats.proportion_confint(wrong[type], right[type]+wrong[type], method='beta', alpha = 0.05) for type in right}
error_rate

{('merged consonants', 'lateral vowels'): (0.12365273084856607,
  0.2304015748565118),
 ('full consonants', 'schwa suppressed vowels'): (0.003744692959841907,
  0.03470059332874382),
 ('full consonants', 'full vowels'): (0.009507546026088847,
  0.05520374109350262),
 ('full consonants', 'flattened lateral vowels'): (0.046683229429337635,
  0.10928166509423455),
 ('merged consonants', 'no vowels'): (0.2603702552555637, 0.36667283997864203),
 ('merged consonants', 'short suppressed vowels'): (0.06729252696279726,
  0.139048044530566),
 ('full consonants', 'long vowels'): (0.051685357443948414,
  0.11421903967718484),
 ('full plosives, merged frictives', 'full vowels'): (0.01919065091251528,
  0.07176499619624185),
 ('full plosives, merged frictives',
  'short suppressed vowels'): (0.02463365525921415, 0.1079896608168007),
 ('full plosives, merged frictives',
  'schwa suppressed vowels'): (0.01000475009508759, 0.04492000238411904),
 ('full plosives, merged frictives', 'no vowels'): (0.104

In [66]:
import statsmodels.stats.weightstats as sw

def ave_bits(type):
    num_alphabet = len(set([z for y in vowel_types[type[1]] + consonant_types[type[0]] if len(y[1]) > 0 for z in y[1]]))
    data = []
    for sent in bank:
        if len(remove_punctuation(sent[0].lower()).split()) != len(remove_punctuation(sent[5].lower()).split()):
            print("ERROR:", sent)
            continue
        if sent[2] != type[0] or sent[3] != type[1]:
            continue
        for word in remove_punctuation(sent[5].lower()).split():
            data.append(np.log2(num_alphabet)*len(word))
    return sw.zconfint(data, alpha=0.05)

In [67]:
ave_bit_conf = {type:ave_bits(type) for type in error_rate}

In [68]:
import plotly.express as px
df = pd.DataFrame([{"type":type[0] + "<br>" + type[1], "mid_bit":(ave_bit_conf[type][1]+ave_bit_conf[type][0])/2, "bit_width":(ave_bit_conf[type][1]-ave_bit_conf[type][0])/2, "mid_error":(error_rate[type][1]+error_rate[type][0])/2, "error_width":(error_rate[type][1]-error_rate[type][0])/2,} for type in ave_bit_conf])

fig = px.scatter(df, x="mid_bit", y="mid_error", text="type",
                 error_x="bit_width", error_y="error_width")

fig.update_layout(width=1280, height=1024,
                  xaxis_title="Average Outline Complexity (bits)",
                  yaxis_title="Reconstruction Error (probability)")
fig.show()

In [69]:
import plotly.express as px
df = pd.DataFrame([{"type":type[0] + "<br>" + type[1], "mid_bit":(ave_bit_conf[type][1]+ave_bit_conf[type][0])/2, "bit_width":(ave_bit_conf[type][1]-ave_bit_conf[type][0])/2, "mid_error":(error_rate[type][1]+error_rate[type][0])/2, "error_width":(error_rate[type][1]-error_rate[type][0])/2,} for type in ave_bit_conf])

fig = px.scatter(df, x="mid_bit", y="mid_error", text="type")

fig.update_layout(width=1280, height=1024,
                  xaxis_title="Average Outline Complexity (bits)",
                  yaxis_title="Reconstruction Error (probability)")
fig.show()

In [70]:
sub_types = [('full consonants', 'schwa suppressed vowels'), ('merged consonants', 'no vowels'), ('merged consonants', 'schwa suppressed vowels'), ('full consonants', 'no vowels')]
#sub_types = [('full consonants', 'schwa suppressed vowels'), ('full plosives, merged frictives', 'schwa suppressed vowels'), ('merged consonants', 'no vowels'), ('merged consonants', 'schwa suppressed vowels'), ('full consonants', 'no vowels'), ('full plosives, merged frictives', 'no vowels')]


import plotly.express as px
df = pd.DataFrame([{"type":type[0] + "<br>" + type[1], "mid_bit":(ave_bit_conf[type][1]+ave_bit_conf[type][0])/2, "bit_width":(ave_bit_conf[type][1]-ave_bit_conf[type][0])/2, "mid_error":(error_rate[type][1]+error_rate[type][0])/2, "error_width":(error_rate[type][1]-error_rate[type][0])/2,} for type in sub_types])

fig = px.scatter(df, x="mid_bit", y="mid_error", text="type",
                 error_x="bit_width", error_y="error_width")

fig.update_layout(width=1280, height=1024,
                  xaxis_title="Average Measured Outline Complexity (bits)",
                  yaxis_title="Measured Human Contextual Reconstruction Error (probability)")
fig.show()

In [71]:
fig.write_image("human_extremes.svg")

In [72]:
with open('folk_system_scores.json', 'r') as file:
        computed_data = json.load(file)

In [73]:
name_alignment = {
    'FC,FV':('full consonants', 'full vowels'),
    'FPMF,FV':('full plosives, merged frictives', 'full vowels'),
    'MC,FV':('merged consonants', 'full vowels'),
    'FC,SSV':('full consonants', 'schwa suppressed vowels'),
    'FPMF,SSV':('full plosives, merged frictives', 'schwa suppressed vowels'),
    'MC,SSV':('merged consonants', 'schwa suppressed vowels'),
    'FC,ShSV':('full consonants', 'short suppressed vowels'),
    'FPMF,ShSV':('full plosives, merged frictives', 'short suppressed vowels'),
    'MC,ShSV':('merged consonants', 'short suppressed vowels'),
    'FC,LoV':('full consonants', 'long vowels'),
    'FPMF,LoV':('full plosives, merged frictives', 'long vowels'),
    'MC,LoV':('merged consonants', 'long vowels'),
    'FC,LV':('full consonants', 'lateral vowels'),
    'FPMF,LV':('full plosives, merged frictives', 'lateral vowels'),
    'MC,LV':('merged consonants', 'lateral vowels'),
    'FC,FLV':('full consonants', 'flattened lateral vowels'),
    'FPMF,FLV':('full plosives, merged frictives', 'flattened lateral vowels'),
    'MC,FLV':('merged consonants', 'flattened lateral vowels'),
    'FC,NV':('full consonants', 'no vowels'),
    'FPMF,NV':('full plosives, merged frictives', 'no vowels'),
    'MC,NV':('merged consonants', 'no vowels')}

[(x[0],x[1]['reconstruction_error'],wrong[name_alignment[x[0]]]/(right[name_alignment[x[0]]]+wrong[name_alignment[x[0]]])) for x in computed_data]

[('FC,FV', 0.03285094362542118, 0.02575107296137339),
 ('FPMF,FV', 0.03711484616306682, 0.03968253968253968),
 ('MC,FV', 0.05103137885885156, 0.05829596412556054),
 ('FC,SSV', 0.025486654793430263, 0.0136986301369863),
 ('FPMF,SSV', 0.02965160691192459, 0.023054755043227664),
 ('MC,SSV', 0.04347157613251906, 0.06313131313131314),
 ('FC,ShSV', 0.06115825262531338, 0.06578947368421052),
 ('FPMF,ShSV', 0.0683186320679362, 0.056338028169014086),
 ('MC,ShSV', 0.09118930364096578, 0.09897610921501707),
 ('FC,LoV', 0.11686952963055652, 0.07886435331230283),
 ('FPMF,LoV', 0.1299224631398923, 0.11764705882352941),
 ('MC,LoV', 0.16594955358722174, 0.11304347826086956),
 ('FC,LV', 0.09893907275029679, 0.08363636363636363),
 ('FPMF,LV', 0.11178819725982148, 0.05339805825242718),
 ('MC,LV', 0.1440412259470797, 0.1722488038277512),
 ('FC,FLV', 0.17558104462061896, 0.07357859531772576),
 ('FPMF,FLV', 0.1929383807408963, 0.12023460410557185),
 ('MC,FLV', 0.22864838598536974, 0.17733990147783252),
 ('F

In [74]:
df_comp = pd.DataFrame([{"type":x[0], "measured_error":wrong[name_alignment[x[0]]]/(right[name_alignment[x[0]]]+wrong[name_alignment[x[0]]]), "computed_error":x[1]['reconstruction_error']} for x in computed_data])

fig = px.scatter(df_comp, x="computed_error", y="measured_error", text="type")

fig.update_layout(width=1280, height=1024,
                  xaxis_title="Reconstruction Error (computed)",
                  yaxis_title="Reconstruction Error (measured)")

fig.show()

In [75]:
def logit(x):
    return np.log(x/(1-x))

def logistic(y):
    return 1/(1+np.exp(-y))

In [76]:
linregress(logit(df_comp['computed_error']), logit(df_comp['measured_error']))

LinregressResult(slope=np.float64(0.8802359497904008), intercept=np.float64(-0.5095902201294891), rvalue=np.float64(0.9099963481191335), pvalue=np.float64(1.0687851699298798e-08), stderr=np.float64(0.09200871931996098), intercept_stderr=np.float64(0.21715480378114485))

$$
\log(p_{human}) = 0.9\log(p)-0.5
$$
implies
$$
p_{human} = 0.6p^{0.9}
$$

In [77]:
df_comp = pd.DataFrame([{"type":x[0], "measured_error":logit(wrong[name_alignment[x[0]]]/(right[name_alignment[x[0]]]+wrong[name_alignment[x[0]]])), "computed_error":logit(x[1]['reconstruction_error'])} for x in computed_data])

fig = px.scatter(df_comp, x="computed_error", y="measured_error", text="type", trendline='ols')

# df_logit = pd.DataFrame([{"x":x,"y":logistic(0.9*logit(x)-0.5)} for x in np.linspace(0,0.4,100)])

fig.update_layout(width=1280, height=1024,
                  xaxis_title="Logit Reconstruction Error (computed)",
                  yaxis_title="Logit Reconstruction Error (measured)")

fig.show()

In [78]:
fig.write_image("human_line_fit.svg")

In [79]:
df_comp = pd.DataFrame([{"type":x[0], "measured_error":wrong[name_alignment[x[0]]]/(right[name_alignment[x[0]]]+wrong[name_alignment[x[0]]]), "computed_error":x[1]['reconstruction_error']} for x in computed_data])

fig = px.scatter(df_comp, x="computed_error", y="measured_error", text="type")

df_logit = pd.DataFrame([{"x":x,"y": logistic(0.9*logit(x)-0.5)} for x in np.linspace(0,0.3,100)])

fig.add_scatter(x=df_logit["x"], y=df_logit["y"], mode='lines', name='Best Fit')

fig.update_layout(width=1280, height=1024,
                  xaxis_title="Reconstruction Error (computed)",
                  yaxis_title="Reconstruction Error (measured)")

fig.show()

/var/folders/ng/pntt6yds68xb60yvl8c33wpm0000gq/T/ipykernel_40721/2823379121.py:2: RuntimeWarning:

divide by zero encountered in log



In [80]:
df_comp = pd.DataFrame([{"type":x[0], "measured_error":wrong[name_alignment[x[0]]]/(right[name_alignment[x[0]]]+wrong[name_alignment[x[0]]]), "computed_error":x[1]['reconstruction_error'],"mid_error":(error_rate[name_alignment[x[0]]][1]+error_rate[name_alignment[x[0]]][0])/2, "error_width":(error_rate[name_alignment[x[0]]][1]-error_rate[name_alignment[x[0]]][0])/2} for x in computed_data])

fig = px.scatter(df_comp, x="computed_error", y="mid_error", error_y="error_width")

df_logit = pd.DataFrame([{"x":x,"y": logistic(0.9*logit(x)-0.5)} for x in np.linspace(0,1.0,100)])

fig.add_scatter(x=df_logit["x"], y=df_logit["y"], mode='lines', name='Best Fit')

fig.update_layout(width=1280, height=1024,
                  xaxis_title="Reconstruction Error (computed)",
                  yaxis_title="Reconstruction Error (measured)")
fig.show()

/var/folders/ng/pntt6yds68xb60yvl8c33wpm0000gq/T/ipykernel_40721/2823379121.py:2: RuntimeWarning:

divide by zero encountered in log

/var/folders/ng/pntt6yds68xb60yvl8c33wpm0000gq/T/ipykernel_40721/2823379121.py:2: RuntimeWarning:

divide by zero encountered in scalar divide



In [81]:
fig.write_image("human_full_range.svg")